In [ ]:
#instalação libs

#%pip install yfinance
#%pip install crewai
#%pip install crewai crewai-tools
#%pip install streamlit




In [13]:
#import libs

import json
import os
from datetime import datetime

import yfinance as yf

from crewai import  Agent, Task, crew, process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown



In [11]:
#CRIANDO YAHOO FINANCE TOOOLS

def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-20", end="2024-08-19")
    return(stock) 

yft = Tool(
name = "Yahoo Finance Tools",
description =" ",
func = lambda ticket: fetch_stock_price(ticket) 


)

In [ ]:
#IMPORTANDO OPENAI

os.environ['OPEN_API_KEY'] = "",
llm = ChatOpenAI(model = "gpt-3.5-turbo") 

In [ ]:
stockPriceAnalyst = Agent(

role ="",
goal="",
backstore = "",
verbose = True,
llm = llm,
max_iter = 5,
memory = True,
tools = [],
allow_delegation = False

)

In [ ]:
get_StockPrice = Task(

description = "",
expected_output = "",

agent = stockPriceAnalyst

)

In [ ]:
#IMPORTANDO A TOOL DE SEARCH 

search_tool = DuckDuckGoSearchResults(backend='news',num_results = 10 )

In [ ]:
stockNewsAnalyst = Agent(

role ="",
goal="",
backstore = "",
verbose = True,
llm = llm,
max_iter = 5,
memory = True,
tools = [search_tool],
allow_delegation = False

)

In [ ]:
get_news = Task(

description = "",
expected_output = "",

agent =  stockNewsAnalyst


)

In [ ]:
stockAnalystWriter = Agent(

role ="",
goal="",
backstore = "",
verbose = True,
llm = llm,
max_iter = 5,
memory = True,
tools = [],
allow_delegation = True   
)

In [ ]:
get_writeAnalyses = Task(

description = " ",
expected_output = " ", 

agent = stockAnalystWriter,
context = [get_StockPrice, get_news]

)

In [ ]:
crew = crew(

    agents = [stockPriceAnalyst,stockNewsAnalyst,stockAnalystWriter],
    tasks = [get_StockPrice,get_news,get_writeAnalyses],
    verbose = 2,
    process = process.hierarchical,
    full_output = True,
    share_crew = False,
    manager_llm = llm,
    max_iter = 15

)

In [ ]:
results = crew.kickoff(inputs= {'ticket': 'AAPL'})

In [ ]:
list(results.keys())

In [ ]:
Markdown(results['final_output'])